## Query EMIT and ENMAP over transect sites

Downloading requires authentication entry in `.netrc` for the NASA Earthdata Login

In [1]:
import pystac_client
import boto3
import requests
from pathlib import Path
import os

from pystac_client import Client
from datetime import datetime, timedelta
import joblib

import pandas as pd
import numpy as np

import json
import joblib

In [2]:
def emit_stac(bbox, datetime_bounds=None, max_cloud=20):
    """
    Query the STAC catalog with the specified bounding box and return matching items.

    :param bbox: Tuple of (swLon, swLat, neLon, neLat) defining the bounding box.
    :return: List of items returned by the query.
    """
    # Define the catalog
    # LPCLOUD catalog URL
    catalog_url = 'https://cmr.earthdata.nasa.gov/cloudstac/LPCLOUD/'
    catalog = Client.open(catalog_url)

    # Define search parameters with the passed bounding box
    search_params = {
        "collections": ["EMITL2ARFL_001"],  # Specify the collection
        "bbox": bbox,                       # Use the provided bounding box
        "query": {
            "eo:cloud_cover": {"lt": max_cloud}   # Query parameter: cloud cover less than x%
        }
    }
    
    if not datetime_bounds is None:
        search_params["datetime"] = datetime_bounds
    
    # Run the STAC query
    query = catalog.search(**search_params)

    # List items returned by the query
    items = list(query.items())
    return items



def enmap_stac(bbox, datetime_bounds=None, max_cloud=20):
    """
    Query the STAC catalog with the specified bounding box and return matching items.

    :param bbox: List of [swLon, swLat, neLon, neLat] defining the bounding box.
    :return: ItemCollection returned by the query.
    """
    # Open the catalog
    catalog_url = "https://geoservice.dlr.de/eoc/ogc/stac/v1/"
    catalog = Client.open(catalog_url)

    # Collections to be searched
    collections = ["ENMAP_HSI_L2A"]

    # Perform the search
    search = catalog.search(
        collections=collections,
        bbox=bbox,
        datetime=datetime_bounds
    )

    # Return the items collection
    items = search.item_collection()
    if len(items) > 0:
        enmap_items = []
        for enmap_item in items:
            cloud_cover = float(enmap_item.properties["eo:cloud_cover"])
            if cloud_cover < max_cloud: # Can't seem to query their stack for cloud cover
                # Add the two items to the matched_items
                enmap_items.append(enmap_item)
        return enmap_items
    else: return []


In [14]:
# Read the CSV file into a DataFrame
df = pd.read_csv('s2_fc/star_transects.csv')

df = df[df.obs_time>'2022-07-01T00:00:00']

In [15]:
def fractionalCoverSatView(df):
    """
    Calculate fractional cover as viewed from a satellite based on intercept data from a DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing the necessary columns:
        - 'num_points', 'unoccluded', 'over_b', 'over_d', 'over_g', 'crn',
          'mid_b', 'mid_g', 'mid_d', 'dead', 'litter', 'crust', 'dist',
          'rock', 'green', 'crypto'.

    Returns:
    tuple of np.ndarray: Six NumPy arrays, each representing one of the calculated cover metrics.
    """
    
    totalPVCover_list = []
    totalNPVCover_list = []
    totalBareCover_list = []
    totalCryptoCover_list = []
    satPersistentGreen_list = []
    persistentNPVFraction_list = []

    for _, row in df.iterrows():
        nTotal = row['num_points']
        nUnoccluded = row['unoccluded']
        
        # Canopy Layer
        nCanopyBranch = row['over_b'] * nTotal / 100.0
        nCanopyDead = row['over_d'] * nTotal / 100.0
        nCanopyGreen = row['over_g'] * nTotal / 100.0
        
        # Midstory Layer
        nMidBranch = row['mid_b'] * nTotal / 100.0
        nMidGreen = row['mid_g'] * nTotal / 100.0
        nMidDead = row['mid_d'] * nTotal / 100.0
        
        # Ground Layer
        nGroundDeadLitter = (row['dead'] + row['litter']) * nUnoccluded / 100.0
        nGroundCrustDistRock = (row['crust'] + row['dist'] + row['rock']) * nUnoccluded / 100.0
        nGroundGreen = row['green'] * nUnoccluded / 100.0
        nGroundCrypto = row['crypto'] * nUnoccluded / 100.0
        
        # Calculations
        canopyFoliageProjectiveCover = nCanopyGreen / (nTotal - nCanopyBranch)
        canopyDeadProjectiveCover = nCanopyDead / (nTotal - nCanopyBranch)
        canopyBranchProjectiveCover = nCanopyBranch / nTotal * (1.0 - canopyFoliageProjectiveCover - canopyDeadProjectiveCover)
        canopyPlantProjectiveCover = (nCanopyGreen+nCanopyDead + nCanopyBranch) / nTotal
        
        midFoliageProjectiveCover = nMidGreen / nTotal
        midDeadProjectiveCover = nMidDead / nTotal
        midBranchProjectiveCover = nMidBranch / nTotal
        midPlantProjectiveCover = (nMidGreen + nMidDead + nMidBranch) / nTotal
        
        satMidFoliageProjectiveCover = midFoliageProjectiveCover * (1 - canopyPlantProjectiveCover)
        satMidDeadProjectiveCover = midDeadProjectiveCover * (1 - canopyPlantProjectiveCover)
        satMidBranchProjectiveCover = midBranchProjectiveCover * (1 - canopyPlantProjectiveCover)
        
        groundPVCover = nGroundGreen / nUnoccluded
        groundNPVCover = nGroundDeadLitter / nUnoccluded
        groundBareCover = nGroundCrustDistRock / nUnoccluded
        groundCryptoCover = nGroundCrypto / nUnoccluded
        groundTotalCover = (nGroundGreen + nGroundDeadLitter + nGroundCrustDistRock) / nUnoccluded
        
        satGroundPVCover = groundPVCover * (1 - midPlantProjectiveCover) * (1 - canopyPlantProjectiveCover)
        satGroundNPVCover = groundNPVCover * ( 1- midPlantProjectiveCover) * (1 - canopyPlantProjectiveCover)
        satGroundBareCover = groundBareCover * (1 - midPlantProjectiveCover) * (1 - canopyPlantProjectiveCover)
        satGroundCryptoCover = groundCryptoCover * (1 - midPlantProjectiveCover) * (1 - canopyPlantProjectiveCover)
        
        totalPVCover = canopyFoliageProjectiveCover + satMidFoliageProjectiveCover + satGroundPVCover
        totalNPVCover = (canopyDeadProjectiveCover + canopyBranchProjectiveCover +
                         satMidDeadProjectiveCover + satMidBranchProjectiveCover + satGroundNPVCover)
        totalBareCover = satGroundBareCover
        totalCryptoCover = satGroundCryptoCover
        
        satPersistentGreen = canopyFoliageProjectiveCover + satMidFoliageProjectiveCover
        persistentNPVFraction = (canopyDeadProjectiveCover + canopyBranchProjectiveCover +
                                 satMidDeadProjectiveCover + satMidBranchProjectiveCover)

        totalPVCover_list.append(totalPVCover)
        totalNPVCover_list.append(totalNPVCover)
        totalBareCover_list.append(totalBareCover)
        totalCryptoCover_list.append(totalCryptoCover)
        satPersistentGreen_list.append(satPersistentGreen)
        persistentNPVFraction_list.append(persistentNPVFraction)

    df['totalPVCover'] = np.array(totalPVCover_list)
    df['totalNPVCover'] = np.array(totalNPVCover_list)
    df['totalBareCover'] = np.array(totalBareCover_list)
    df['totalCryptoCover'] = np.array(totalCryptoCover_list)
    df['satPersistentGreen'] = np.array(satPersistentGreen_list)
    df['persistentNPVFraction'] = np.array(persistentNPVFraction_list)
    return (
        np.array(totalPVCover_list),
        np.array(totalNPVCover_list),
        np.array(totalBareCover_list),
        np.array(totalCryptoCover_list),
        np.array(satPersistentGreen_list),
        np.array(persistentNPVFraction_list),
    )

# Calculate fractional cover as viewed from a satellite based on intercept data   
totalPVCover,totalNPVCover,totalBareCover,totalCryptoCover,satPersistentGreen,persistentNPVFraction = fractionalCoverSatView(df)

df_fc = df[['FID', 'ref_x', 'ref_y', 'obs_time', 'totalPVCover','totalNPVCover','totalBareCover','totalCryptoCover', 'satPersistentGreen', 'persistentNPVFraction']]


In [ ]:
len(df_fc)

In [ ]:
df_fc.head()

In [ ]:
# Iterate over each row in the DataFrame with a progress bar
emit_all = {}
#enmap_all = {}

for _, row in df_fc.iterrows():
    print(_)
    longitude = row['ref_x']
    latitude = row['ref_y']
    radius = 0.001 #deg, ~ 100m

    # Parse the observation time string into a datetime object
    date = datetime.strptime(row['obs_time'], "%Y-%m-%dT%H:%M:%S")
    start_datetime = date - timedelta(days=14)
    end_datetime = date + timedelta(days=14)
    stack_datetime = f"{start_datetime.strftime('%Y-%m-%dT%H:%M:%S')}Z/{end_datetime.strftime('%Y-%m-%dT%H:%M:%S')}Z"

    bounding_box = (longitude-radius, latitude-radius, longitude+radius, latitude+radius)

    emit_items = emit_stac(bounding_box, datetime_bounds=stack_datetime, max_cloud=30)
    if len(emit_items)>0:
        emit_all[row['FID']]= emit_items
    #if len(emit_items) ==1: 
    #    emit_all[row['FID']]= emit_items[0]
    #elif len(emit_items)>1:
    #    closest = emit_items[0]
    #    timediff = abs(datetime.strptime(emit_items[0].properties['datetime'], "%Y-%m-%dT%H:%M:%S.%fZ")-date)
    #    for item in emit_items[1:]:
    #        if abs(datetime.strptime(item.properties['datetime'], "%Y-%m-%dT%H:%M:%S.%fZ")-date)<timediff:
    #            closest=item
    #    emit_all[row['FID']]= closest

    #enmap_items = enmap_stac(bounding_box, datetime_bounds=stack_datetime, max_cloud=20)
    #if len(enmap_items) >0: enmap_all[row['FID']]= enmap_items
                
joblib.dump(emit_all, 'transect_emit_all_multiple.pkl')

In [ ]:
for fid, items in emit_all.items():
    print(df_fc[df_fc['FID']==fid][['FID', 'obs_time','totalPVCover','totalNPVCover','totalBareCover']])

In [ ]:
# Iterate over each row in the DataFrame with a progress bar
enmap_all = {}

for _, row in df_fc.iterrows():
    print(_)
    longitude = row['ref_x']
    latitude = row['ref_y']
    radius = 0.001 #deg, ~ 100m

    # Parse the observation time string into a datetime object
    date = datetime.strptime(row['obs_time'], "%Y-%m-%dT%H:%M:%S")
    start_datetime = date - timedelta(days=14)
    end_datetime = date + timedelta(days=14)
    stack_datetime = f"{start_datetime.strftime('%Y-%m-%dT%H:%M:%S')}Z/{end_datetime.strftime('%Y-%m-%dT%H:%M:%S')}Z"

    bounding_box = (longitude-radius, latitude-radius, longitude+radius, latitude+radius)

    enmap_items = enmap_stac(bounding_box, datetime_bounds=stack_datetime, max_cloud=50)
    if len(enmap_items) >0: enmap_all[row['FID']]= enmap_items
                
    

In [ ]:
enmap_all

In [ ]:
def create_geojson_feature_collection(matched_items, df=df_fc, type="emit"):
    features = []

    for fid, items in matched_items.items():
        for item in items:
            emit_feature = {
                "type": "Feature",
                "geometry": item.geometry,
                "properties": {
                    "id": item.id,
                    "datetime": item.properties["datetime"],
                    "cloud_cover": float(item.properties["eo:cloud_cover"]),
                    "type": type
                }
            }
            features.append(emit_feature)

        row =df_fc[df_fc['FID']==fid]
        geometry = {"type": "Point", "coordinates": [row['ref_x'].values[0], row['ref_y'].values[0]]}
        transect_feature = {
            "type": "Feature",
            "geometry": geometry,
            "properties": {
                "id": fid,
                "datetime": row['obs_time'].values[0],
                "totalPVCover": row['totalPVCover'].values[0],
                "totalNPVCover": row['totalNPVCover'].values[0],
                "totalBareCover":row['totalBareCover'].values[0],
                "type": 'transect'
            }
        }        
        features.append(transect_feature)   
 
    feature_collection = {
        "type": "FeatureCollection",
        "features": features
    }

    return feature_collection

geojson_feature_collection = create_geojson_feature_collection(emit_all)

with open('transect_matched_features.geojson', 'w') as f:
    json.dump(geojson_feature_collection, f, indent=2)

### Download data

In [6]:
def download_emit(url, local_filename):
    local_file = Path(local_filename)
    # Get the directory path
    local_path = local_file.parent
    # Create the directory and intermediate directories if needed
    local_path.mkdir(parents=True, exist_ok=True)
    
    try:
        # Perform the GET request (requests will use .netrc for credentials)
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Save the file locally
            with open(local_file, "wb") as file:
                file.write(response.content)
            print(f"File downloaded successfully: {local_file}")
        else:
            print(f"Failed to download file. Status code: {response.status_code}")
            print(f"Response text: {response.text}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [9]:
# Download data

local_path = 'data/transect/'

for fid, items in emit_all.items():
    for item in items:
        for key, asset in item.assets.items():
            if asset.roles==['data']:
                local_file = asset.href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', local_path)
                if not os.path.exists(local_file): download_emit(asset.href, local_file)
                else: print("File exists:", local_file)
            if asset.roles==['browse']:
                local_file = asset.href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/', local_path)
                if not os.path.exists(local_file): download_emit(asset.href, local_file)
                else: print("File exists:", local_file)
    

File exists: data/transect/EMITL2ARFL.001/EMIT_L2A_RFL_001_20231030T024137_2330302_025/EMIT_L2A_RFL_001_20231030T024137_2330302_025.png
File exists: data/transect/EMITL2ARFL.001/EMIT_L2A_RFL_001_20231030T024137_2330302_025/EMIT_L2A_RFL_001_20231030T024137_2330302_025.nc
File exists: data/transect/EMITL2ARFL.001/EMIT_L2A_RFL_001_20231030T024137_2330302_025/EMIT_L2A_RFLUNCERT_001_20231030T024137_2330302_025.nc
File exists: data/transect/EMITL2ARFL.001/EMIT_L2A_RFL_001_20231030T024137_2330302_025/EMIT_L2A_MASK_001_20231030T024137_2330302_025.nc
File exists: data/transect/EMITL2ARFL.001/EMIT_L2A_RFL_001_20231103T010500_2330701_022/EMIT_L2A_RFL_001_20231103T010500_2330701_022.png
File exists: data/transect/EMITL2ARFL.001/EMIT_L2A_RFL_001_20231103T010500_2330701_022/EMIT_L2A_RFL_001_20231103T010500_2330701_022.nc
File exists: data/transect/EMITL2ARFL.001/EMIT_L2A_RFL_001_20231103T010500_2330701_022/EMIT_L2A_RFLUNCERT_001_20231103T010500_2330701_022.nc
File exists: data/transect/EMITL2ARFL.00

## Match over transect

In [10]:
emit_all = joblib.load('transect_match/transect_emit_all_multiple.pkl')

In [11]:
from tools.emit_tools_new import emit_xarray, band_mask

In [12]:
def emit_save_to_geotiff(ds_masked, geotiff_path):
    """
    Save masked EMIT xarray to a geotiff file with bands names as interger central wavelengths
    """
    if not os.path.exists(geotiff_path):
        wave_int = ds_masked.coords["bands"].values.astype(int).astype(str)
        ds_masked.reflectance.to_dataset(dim='bands').drop_dims('bands').rename(dict(zip(ds_masked.coords["bands"].values, wave_int))).rio.to_raster(geotiff_path, tiled=True, windowed=True)
    

In [ ]:
emit_all

In [16]:
local_path = Path('data/transect/EMITL2ARFL.001/')
transect_spectra = {}

for fid, items in emit_all.items():
    for item in items:
        row =df_fc[df_fc['FID']==fid]
        output_csv = local_path/(fid+'_'+df_fc[df_fc['FID']==fid].obs_time.values[0][:10].replace('-','')+'_'+item.id+'.csv')

        rfl_path = local_path/item.id/(item.id+'.nc')
        mask_path = rfl_path.parent / rfl_path.name.replace('_RFL_', '_MASK_')
        ds_mask = band_mask(mask_path)
        ds_masked = emit_xarray(rfl_path, unpacked_bmask= ds_mask)

        emit_save_to_geotiff(ds_masked, geotiff_path=str(rfl_path).replace('.nc', '_masked.tif'))

        df = ds_masked.reflectance.sel(latitude = row['ref_y'].values[0], longitude = row['ref_x'].values[0], method="nearest").where(ds_masked.good_wavelengths).to_dataframe()
        df.to_csv(output_csv)
        transect_spectra[(fid, item.id)] = df

In [17]:
joblib.dump(transect_spectra, 'transect_match/transect_emit_spectra_multiple.pkl')

['transect_match/transect_emit_spectra_multiple.pkl']

In [18]:
transect_spectra

{('star_transects.fid-35735e87_1942a663648_-754d',
  'EMIT_L2A_RFL_001_20231030T024137_2330302_025'):              wavelengths   fwhm  good_wavelengths   latitude   longitude  \
 bands                                                                      
 381.005585    381.005585  8.415               1.0 -26.919186  145.507549   
 388.409210    388.409210  8.415               1.0 -26.919186  145.507549   
 395.815826    395.815826  8.415               1.0 -26.919186  145.507549   
 403.225403    403.225403  8.415               1.0 -26.919186  145.507549   
 410.638000    410.638000  8.417               1.0 -26.919186  145.507549   
 ...                  ...    ...               ...        ...         ...   
 2463.381592  2463.381592  8.803               1.0 -26.919186  145.507549   
 2470.767822  2470.767822  8.804               1.0 -26.919186  145.507549   
 2478.153076  2478.153076  8.806               1.0 -26.919186  145.507549   
 2485.538574  2485.538574  8.807               1.0 -